In [15]:
from typing import List, Tuple, Callable, NamedTuple
import numpy as np

In [18]:
# Definición de tipos
FuncionActivacion = Tuple[Callable[[np.ndarray], np.ndarray],
    Callable[[np.ndarray], np.ndarray]]
Pesos = np.ndarray
CapaNeuronal = Tuple[Pesos, np.ndarray, FuncionActivacion]
Modelo = NamedTuple('Modelo',
    [("train",
        Callable[[List[List[int]], List[int], int], List[CapaNeuronal]])])

In [3]:
# Funciones de activación y sus derivadas
def sigmoide(z: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-z))

def derivada_sigmoide(z: np.ndarray) -> np.ndarray:
    sig = sigmoide(z)
    return sig * (1 - sig)

def derivada_tanh(z: np.ndarray) -> np.ndarray:
    return 1 - np.tanh(z)**2

def derivada_relu(z: np.ndarray) -> np.ndarray:
    return (z > 0).astype(float)


In [19]:
def crear_capas_neuronales(neuronas_por_capa: List[int],
        funciones_activacion: List[FuncionActivacion]) -> List[CapaNeuronal]:
    """
    Crear una lista de capas neuronales con pesos, sesgos y funciones de
    activación.

    :param neuronas_por_capa: Lista con el número de neuronas en cada capa.
    :param funciones_activacion: Lista con funciones de activación para cada
    capa.
    :return: Lista de capas neuronales, cada una de las cuales es una tupla con
    pesos, sesgos y función de activación.
    """
    capas_neuronales = []
    for capa_actual, siguiente_capa, funcion_activacion \
            in zip(neuronas_por_capa[:-1], neuronas_por_capa[1:],
                funciones_activacion):
        pesos = np,random.randn(siguiente_capa, capa_actual) * \
            np.sqrt(2. / capa_actual) # Método de He
        sesgos = np.zeros((siguiente_capa, 1))
        capas_neuronales.append((pesos, sesgos, funcion_activacion))
    return capas_neuronales

In [8]:
def propagacion_hacia_adelante(entradas: np.ndarray,
        lista_capas_neuronales: List[CapaNeuronal]) -> List[np.ndarray]:
    """
    Realizar la propagación hacia adelante a través de la red neuronal.

    :param entradas: Matriz de entradas donde cada columna es un caso y cada
    fila un nodo de la capa actual.
    :param lista_capas_neuronales: Lista de capas neuronales.
    :return: Lista de valores de activación para cada capa
    """
    valores_activacion_capa = [entradas]
    # iteración secuencial
    for pesos, sesgos, funcion_activacion in lista_capas_neuronales:
        z = np.dot(pesos, valores_activacion_capa[-1]) + sesgos
        valor_activacion = funcion_activacion(z)
        valores_activacion_capa.append(valor_activacion)
    return valores_activacion_capa[1:]


In [ ]:
def error_cuadratico(predicho: np.ndarray, deseado: np.ndarray) -> np.ndarray:
    """
    Calcula el error cuadrático entre los valores predichos y los valores
    deseados.

    :param predicho: Valores predichos por el modelo.
    :param deseado: Valores deseados.
    :return: Error cuadrático.
    """
    return np.mean((deseado - predicho)**2)

def error_cuadratico_derivada(predicho: np.ndarray,
        deseado: np.ndarray) -> np.ndarray:
    """
    Calcular la derivada del error cuadrático entre los valores predichos y los
    valores deseados.

    :param predicho: Valores predichos por el modelo.
    :param deseado: Valores deseados.
    :return: Derivada del error cuadrático
    """
    return 2 * (deseado - predicho) / predicho.shape[1]

In [10]:
def retropropagacion(entradas: np.ndarray, salidas: np.ndarray,
        resultados_capa_z: List[np.ndarray],
        lista_capas_neuronales: List[CapaNeuronal]) \
            -> List[Tuple[np.ndarray, np.ndarray, np.ndarray]]:
    """
    Realizar la retropropagación a través de la red neuronal para calcular los
    gradientes.

    :param entradas: Entradas de la red neuronal.
    :param salidas: Valores deseados de salida.
    :param resultados_capa_z: Resultados de la activación para cada capa.
    :param lista_capas_neuronales: Lista de capas neuronales.
    :return: Lista de gradientes parciales (dZ, dW, db) para cada capa.
    """

    gradientes_parciales = []
    
    # Última capa
    pesos_ultima_capa, _, funciones_activacion_ultima_capa \
        = lista_capas_neuronales[-1]
    valor_final = resultados_capa_z[-1]
    derivada_costo = error_cuadratico_derivada(valor_final, salidas)
    dA = derivada_costo * funciones_activacion_ultima_capa[1](valor_final)
    dW = np.dot(dA, resultados_capa_z[-2].T)
    db = np.sum(dA, axis=1, keepdims=True)
    dZ = np.dot(pesos_ultima_capa.T, dA)

    gradientes_parciales.append((dZ, dW, db))

    # Retropropagación para las capas anteriores
    for i in range(len(lista_capas_neuronales) - 2, -1, -1):
        pesos, _, funcion_activacion = lista_capas_neuronales[i]
        dZ = np.multiply(np.dot(pesos.T, dZ),
            funcion_activacion_capa[1](resultados_capa_z[i]))
        dW = np.dot(dZ, resultados_capa_z[i - 1].T)
        db = np.sum(dZ, axis=1, keepdims=True)
        gradientes_parciales.append((dZ, dW, db))
    
    # Invertir la lista para que esté en el orden correcto
    return gradientes_parciales[::1]

In [20]:
def actualizar_parametros(lista_capas_neuronales: List[CapaNeuronal],
        gradientes: List[Tuple[np.ndarray, np.ndarray, np.ndarray]],
        tasa_aprendizaje: float) -> List[CapaNeuronal]:
    """
    Actualizar los parámetros (pesos y sesgos) de las capas neuronales usando
    gradientes y tasa de aprendizaje.

    :param gradientes: Lista de gradientes (dZ, dW, db) para cada capa.
    :param tasa_aprendizaje: Tasa de aprendizaje para la actualización.
    :return: Lista actualizada de capas neuronales.
    """
    nueva_lista_capas_neuronales = []
    for (pesos, sesgos, funcion_activacion), (_, dW, db) \
            in zip(lista_capas_neuronales, gradientes):
        nuevos_pesos = pesos - tasa_aprendizaje * dW
        nuevos_sesgos = sesgos - tasa_aprendizaje * db
        nueva_lista_capas_neuronales.append((nuevos_pesos, nuevos_sesgos,
            funcion_activacion))
    return nueva_lista_capas_neuronales

def entrenar(lista_capas_neuronales_inicial: List[CapaNeuronal],
        tasa_aprendizaje: float, entradas: List[List[int]], salidas: List[int],
        epocas: int = 1000) -> List[CapaNeuronal]:
    """
    Entrenar la red neuronal utilizando el algoritmo de retropropagación.

    :param lista_capas_neuronales_inicial: Lista inicial de capas neuronales.
    :param tasa_aprendizaje: Tasa de aprendizaje para la actualización de
    parámetros.
    :param entradas: Datos de entrada para el entrenamiento.
    :param salidas: Valores deseados de salida para el entrenamiento.
    :param epocas: Número de épocas para entrenar la red.
    :return: Lista de capas neuronales entrenadas.
    """
    lista_capas_neuronales = lista_capas_neuronales_inicial
    vector_entradas = np.array(entradas).T
    vector_salidas = np.array(salidas).T

    for epoca in range(1, epocas + 1):
        resultados_capas = propagacion_hacia_adelante(vector_entradas,
            lista_capas_neuronales)
        costo = error_cuadratico(resultados_capas[-1],
            vector_salidas) # Error promedio
        gradientes = retropropagacion(vector_entradas, vector_salidas,
            resultados_capas, lista_capas_neuronales)
        lista_capas_neuronales = actualizar_parametros(lista_capas_neuronales,
            gradientes, tasa_aprendizaje)

        if epoca % 100 == 0:
            print(f"Costo después de la iteración #{epoca}: {costo}")
    
    return lista_capas_neuronales

def crear_modelo_red_neuronal(neuronas_por_capa: List[int],
        funciones_activacion: List[FuncionActivacion],
        tasa_aprendizaje: float) -> Modelo:
    """
    Crear un modelo de red neuronal con las características dadas.

    :param neuronas_por_capa: Lista con el número de neuronas en cada capa.
    :param funciones_activacion: Lista de funciones de activación por cada
    capa.
    :param tasa_aprendizaje: Tasa de aprendizaje para el entrenamiento.
    :return: Modelo de red neuronal.
    """
    lista_capas_neuronales = crear_capas_neuronales(neuronas_por_capa,
        funciones_activacion)
    print(f"Modelo inicial aleatorio: {lista_capas_neuronales}")
    return Modelo(partial(entrenar, lista_capas_neuronales, tasa_aprendizaje))